<a href="https://colab.research.google.com/github/yuvrajtu/deeplearning/blob/master/ZSCHallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
import pandas as pd
df=pd.read_csv("/content/drive/My Drive/zschallenge/data.csv")
feature_all=['match_event_id',           
'location_x',               
'location_y',                
'remaining_min',             
'power_of_shot',             
'knockout_match',            
'game_season',               
'remaining_sec',             
'distance_of_shot',          
'is_goal',                   
'area_of_shot',              
'shot_basics',               
'range_of_shot',            
'team_name',                
'date_of_game',              
'home/away',                
'shot_id_number',           
'lat/lng',                 
'type_of_shot',             
'type_of_combined_shot',   
'match_id',                     
'team_id']
df_new=df[feature_all]
df_new.tail()

# df2=pd.read_csv("/content/drive/My Drive/zschallenge/sample_submission.csv")
# df2.head(10)

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,team_name,date_of_game,home/away,shot_id_number,lat/lng,type_of_shot,type_of_combined_shot,match_id,team_id
30692,397.0,1.0,48.0,6.0,4.0,1.0,1999-00,5.0,24.0,0.0,Center(C),NaN,Less Than 8 ft.,Manchester United,2000-06-19,MANU vs. IND,NaN,"42.982923, -71.446094",shot - 1,NaN,49900088,1610612747
30693,398.0,0.0,0.0,6.0,4.0,1.0,1999-00,5.0,20.0,NaN,Center(C),Goal Area,Less Than 8 ft.,Manchester United,2000-06-19,MANU vs. IND,30694.0,"42.982923, -71.446094",shot - 49,NaN,49900088,1610612747
30694,426.0,-134.0,166.0,3.0,4.0,1.0,1999-00,28.0,41.0,1.0,Left Side Center(LC),Mid Range,16-24 ft.,Manchester United,2000-06-19,MANU vs. IND,30695.0,NaN,NaN,shot - 3,49900088,1610612747
30695,448.0,31.0,267.0,2.0,4.0,1.0,1999-00,10.0,46.0,0.0,Center(C),Penalty Spot,NaN,Manchester United,2000-06-19,MANU vs. IND,30696.0,"42.982923, -71.446094",shot - 26,NaN,49900088,1610612747
30696,471.0,1.0,NaN,0.0,4.0,1.0,1999-00,39.0,27.0,0.0,Center(C),Goal Line,Less Than 8 ft.,Manchester United,2000-06-19,MANU vs. IND,30697.0,"42.982923, -71.446094",shot - 45,NaN,49900088,1610612747


In [0]:
# df.groupby(['is_goal']).match_event_id.count()
# df.isnull().tail()
# df.notnull().tail()
df.isnull().sum()
# df.shape
df_new=df_new[df_new.is_goal.notnull()]
# test.isnull().sum()
# def pivot_feature(feature):
#     pivot = df_new.pivot_table(index=feature,columns='is_goal',values='shot_id_number',aggfunc='count',margins=True)
#     pivot.columns=["is_goal","Not","total"]
#     pivot['SR']=pivot.is_goal/pivot.total
#     return pivot
# pivot_feature('distance_of_shot')

In [139]:
features=['location_x','location_y','remaining_min','power_of_shot','remaining_sec','distance_of_shot','area_of_shot','shot_basics','range_of_shot','type_of_shot','type_of_combined_shot']
dataset=df_new[features]
dataset['location_x'].fillna(value=-245,inplace=True)
dataset['location_y'].fillna(value=-39.0,inplace=True)
dataset['power_of_shot'].fillna(value=7.0,inplace=True)
dataset['remaining_sec'].fillna(value=11.0,inplace=True)
dataset['remaining_min'].fillna(value=7.0,inplace=True)
dataset['distance_of_shot'].fillna(value=28.0,inplace=True)
dataset['area_of_shot'].fillna(value='Center(C)',inplace=True)
dataset['shot_basics'].fillna(value='Goal Line',inplace=True)
dataset['range_of_shot'].fillna(value='16-24 ft.',inplace=True)
dataset['type_of_shot'].fillna(value='shot-1',inplace=True)
dataset['type_of_combined_shot'].fillna(value='shot-2',inplace=True)
dataset.isnull().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


location_x               0
location_y               0
remaining_min            0
power_of_shot            0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
type_of_shot             0
type_of_combined_shot    0
dtype: int64

In [140]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
dataset['area_of_shot'] = labelencoder_X.fit_transform(dataset['area_of_shot'])
dataset['shot_basics'] = labelencoder_X.fit_transform(dataset['shot_basics'])
dataset['type_of_shot'] = labelencoder_X.fit_transform(dataset['type_of_shot'])
dataset['range_of_shot'] = labelencoder_X.fit_transform(dataset['range_of_shot'])
dataset['type_of_combined_shot'] = labelencoder_X.fit_transform(dataset['type_of_combined_shot'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [141]:
dataset.head(10)

,location_x,location_y,remaining_min,power_of_shot,remaining_sec,distance_of_shot,area_of_shot,shot_basics,range_of_shot,type_of_shot,type_of_combined_shot
1,-157.0,0.0,10.0,1.0,22.0,35.0,2,4,2,40,6
2,-101.0,135.0,7.0,1.0,45.0,36.0,1,4,0,18,6
3,138.0,175.0,6.0,1.0,52.0,42.0,4,4,0,57,3
4,0.0,0.0,7.0,2.0,19.0,20.0,0,0,4,57,1
5,-145.0,-11.0,9.0,3.0,32.0,34.0,2,4,2,9,6
6,0.0,0.0,8.0,7.0,52.0,20.0,0,0,4,57,4
8,-65.0,-39.0,6.0,3.0,12.0,32.0,2,1,2,30,6
9,-33.0,-39.0,3.0,3.0,36.0,32.0,0,1,2,39,6
10,-94.0,238.0,1.0,3.0,56.0,45.0,1,5,1,54,6
11,121.0,127.0,11.0,1.0,0.0,37.0,4,4,0,57,3


In [142]:
from sklearn.model_selection import train_test_split
X=dataset
Y=df_new.is_goal
print(Y.isnull().sum())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 100)

0


In [143]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 12, criterion = 'entropy', random_state = 1337)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=12,
                       n_jobs=None, oob_score=False, random_state=1337,
                       verbose=0, warm_start=False)

In [100]:
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score(y_test,y_pred)

0.5934839554682384

In [152]:
df_testor=df[df.is_goal.isnull()]
dataset=df_testor[features]
dataset['shot_id_number']=df_testor['shot_id_number']
dataset['location_x'].fillna(value=-245,inplace=True)
dataset['location_y'].fillna(value=-39.0,inplace=True)
dataset['power_of_shot'].fillna(value=7.0,inplace=True)
dataset['remaining_sec'].fillna(value=11.0,inplace=True)
dataset['remaining_min'].fillna(value=7.0,inplace=True)
dataset['distance_of_shot'].fillna(value=28.0,inplace=True)
dataset['area_of_shot'].fillna(value='Center(C)',inplace=True)
dataset['shot_basics'].fillna(value='Goal Line',inplace=True)
dataset['range_of_shot'].fillna(value='16-24 ft.',inplace=True)
dataset['type_of_shot'].fillna(value='shot-1',inplace=True)
dataset['type_of_combined_shot'].fillna(value='shot-2',inplace=True)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
dataset['area_of_shot'] = labelencoder_X.fit_transform(dataset['area_of_shot'])
dataset['shot_basics'] = labelencoder_X.fit_transform(dataset['shot_basics'])
dataset['type_of_shot'] = labelencoder_X.fit_transform(dataset['type_of_shot'])
dataset['range_of_shot'] = labelencoder_X.fit_transform(dataset['range_of_shot'])
dataset['type_of_combined_shot'] = labelencoder_X.fit_transform(dataset['type_of_combined_shot'])
dataset.shape
dataset['is_goal']=model.predict(dataset[features])
dataset=dataset.dropna(how='any')
dataset.isnull().sum()
dataset[['shot_id_number','is_goal']].to_csv('/content/drive/My Drive/zschallenge/submission2.csv',index=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py